In [90]:
import pymysql
import pandas as pd
# Database connection configuration
db_config = {
    'host': '140.238.230.93',
    'user': 'kartik',
    'password': 'Kartik@24082003',
    'database': 'zabbix',
    'cursorclass': pymysql.cursors.DictCursor  # optional, returns dict rows
}

# SQL Query
sql_query = """
            SELECT
                h.name AS host,
                t.description AS trigger_name,
                e.name AS event_name,
                e.eventid,
                e.acknowledged,
                e.clock AS start_time,
                COALESCE(er.clock, UNIX_TIMESTAMP()) AS end_time,
                COALESCE(er.clock, UNIX_TIMESTAMP()) - e.clock AS duration,
                er.eventid AS recovery_eventid
            FROM hosts h
            JOIN items i ON i.hostid = h.hostid AND i.status = 0
            JOIN functions f ON f.itemid = i.itemid
            JOIN triggers t ON t.triggerid = f.triggerid AND t.status = 0
            JOIN events e ON e.objectid = t.triggerid AND e.object = 0 AND e.value = 1
            LEFT JOIN event_recovery erc ON erc.eventid = e.eventid
            LEFT JOIN events er ON er.eventid = erc.r_eventid
            WHERE 
                h.status = 0
                AND h.flags IN (0, 4)
"""

try:
    # Connect to the database
    connection = pymysql.connect(**db_config)
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)
            results = cursor.fetchall()


except Exception as e:
    print("Error:", e)


In [91]:
df = pd.DataFrame(results)
print(df)
print(df.columns.tolist())

                 host                                       trigger_name  \
0       Zabbix server                     {HOST.NAME} has been restarted   
1       Zabbix server       Utilization of http poller processes is high   
2       Zabbix server            Utilization of poller processes is high   
3       Zabbix server      Number of installed packages has been changed   
4       Zabbix server      Number of installed packages has been changed   
...               ...                                                ...   
5296    Zabbix server                               High CPU utilization   
5297  WIN-EVKHD7M05S9         "spice-agent" (Spice Agent) is not running   
5298  WIN-EVKHD7M05S9                    Active checks are not available   
5299    Zabbix server                               High CPU utilization   
5300    Zabbix server  More than 100 items having missing data for mo...   

                                             event_name  eventid  \
0       Zabbix serv

In [92]:
def get_host_by_group(host_group):
    """
    Function to get the host group for a given host.
    This is a placeholder function; implement the actual logic as needed.
    """
    query = """
    SELECT 
  g.name AS group_name,
  h.host AS host_name
FROM hstgrp g
JOIN hosts_groups hg ON g.groupid = hg.groupid
JOIN hosts h ON hg.hostid = h.hostid
WHERE g.name = %s
"""
    try:
        connection = pymysql.connect(**db_config)
        with connection:
            with connection.cursor() as cursor:
                cursor.execute(query, (host_group,))
                result = cursor.fetchall()
                if result:
                    return result
    
    except Exception as e:
        print("Error retrieving host group:", e)

    # Example logic, replace with actual implementation
    return "Default Group"  # Replace with actual group retrieval logic

In [93]:
host = get_host_by_group("Zabbix servers")

# print(host)  # Print the result of the function call
print([name['host_name'] for name in host])  # Print column names
# print(host['host_name'])  # Example usage of the function

['Zabbix server', 'test_heena', 'test_heena_flag4', 'template_created']


In [ ]:
def get_alerts(time_from: int = None, time_to: int = None, hostname: str = None, limit: int = 100000, host_group: str = None):
    alerts = pd.DataFrame(results)
    alerts = alerts.sort_values(by='start_time', ascending=False)
    al1 = alerts
    if time_from is not None:
        alerts = alerts[alerts['start_time'] >= time_from]
    if time_to is not None:
        alerts = alerts[alerts['start_time'] <= time_to]
    if hostname is not None:
        alerts = alerts[alerts['host'] == hostname]
    if host_group is not None:
        host_list = get_host_by_group(host_group)
        host_in_group = [h['host_name'] for h in host_list]
        print(f"Hosts in group {host_group}: {host_in_group}")
        alerts = alerts[alerts['host'].isin(host_in_group)]

    if limit is not None:
        alerts = alerts.head(limit)

    return alerts


In [98]:
al = get_alerts(host_group="Zabbix servers",time_from=1748443658)
al1 = get_alerts(time_from=1748443658)

Hosts in group Zabbix servers: ['Zabbix server', 'test_heena', 'test_heena_flag4', 'template_created']


In [99]:
print(al)
print(al.shape)


               host                                       trigger_name  \
5300  Zabbix server  More than 100 items having missing data for mo...   
5299  Zabbix server                               High CPU utilization   
5296  Zabbix server                               High CPU utilization   
5295  Zabbix server                               High CPU utilization   
5294  Zabbix server                               High CPU utilization   
5293  Zabbix server                               High CPU utilization   
5292  Zabbix server                               High CPU utilization   
5291  Zabbix server                               High CPU utilization   
5290  Zabbix server                               High CPU utilization   
5289  Zabbix server                               High CPU utilization   
5288  Zabbix server                               High CPU utilization   
5287  Zabbix server                               High CPU utilization   
5286  Zabbix server                   

In [100]:
print(al.columns.tolist())
print(al['host'].unique())
print(al1['host'].unique())  # Print unique hosts in the alerts DataFrame

['host', 'trigger_name', 'event_name', 'eventid', 'acknowledged', 'start_time', 'end_time', 'duration', 'recovery_eventid']
['Zabbix server']
['Zabbix server' 'WIN-EVKHD7M05S9']


In [101]:
if 1748443658>1740657734:
    print("Time condition met: 1748443658 is greater than 1740657734")
    

Time condition met: 1748443658 is greater than 1740657734
